### ArcGIS Administration at Scale
## ESRI Southeast User Conference 2018
## Seth Peery, Sr. GIS Architect, Virginia Tech
### May 3, 2018

This presentation is given as a jupyter notebook. Jupyter notebooks are a web-based interactive Python development environment.
![Architecture](jupyterarch.png)

We don't have to run a Jupyter notebook server, though.  You should have the ArcGIS Python API installed already if you have ArcGIS Pro.  See https://developers.arcgis.com/python/guide/using-the-jupyter-notebook-environment/ 

In [ ]:
# That means we can run code.   Like this:
from arcgis import *
gis = GIS()
map = gis.map("Charlotte Convention Center, Charlotte, NC")
map

# Key concepts

# Rationale for org management
* Software as a Service products must be managed with the same attention we give on-premises systems
* Management objective is to lower impediments to use of AGO
* AGOL depends on finite shared resources; org administration is the stewardship of these resources
 * named users
 * service credits
 * Pro and other licenses
 * usability and organization of the site
* AGOL orgs can become unwieldy at scale
* User lifecycle stages require different management practices over time
* Processes, training and automation become increasingly important for large orgs


# Org administration tasks
* Enterprise Logins
* Auto provisioning
* Initial default privileges and entitements
* Credit stewardship
* Pro Licenses
* Ad hoc requests
* Content migration
* Deprovisioning

![events](adminflow.png)

## The Python API for organization administration
The python API provides a set of objects for administering your Web GIS.
![GIS Module](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_gis_module_01.png)

## Connecting to your Web GIS (ArcGIS Online / ArcGIS Enterprise)
To connect to your organization, we import the requisite libraries and then create an object of type "GIS":

In [ ]:
from arcgis import *
import requests
import time
import csv
import json
import pandas
from time import strftime

### Syntax to connect to ArcGIS Online
` gis = GIS("https://myorganization.maps.arcgis.com",username="An_admin_user",password="Please_do_not_put_this_in_clear_text")   ` 

In my case I put org-specific stuff into a config file so this notebook can be more easily shared with others.
The file looks like this:

`{
    "agolOrg":{
            "url":"https://yourOrgShortName.maps.arcgis.com",
            "username":"*******,
            "password":"*******",
            "shortName":"yourOrgShortName"
    },
    "authService":{
            "url":"https://some_url_that_checks_usernames",
            "username":"*****",
            "password":"*****",
            "valueIfTrue":"true"
    }
}`

### Making the Connection to your Org

In [ ]:

# Then load it up like this
with open('orgConfig.json') as configFile:
    myConfig = json.load(configFile)

# now connect
gis = GIS(myConfig['agolOrg']['url'],username=myConfig['agolOrg']['username'],password=myConfig['agolOrg']['password'])    

# verify that it works
try:
    org = gis.properties.name
    print ("Connected to " + org)
except exception as ex:
        print ("Error retrieving AGOL org properties.")

Now let's get some info about our users.

In [ ]:
userList = []
users = gis.users.search(max_users=20)

for user in users:
    #These things come straight from the user dict
    d_esriUsername = user.username
    d_fullName = user.fullName
    d_email = user.email
    d_role = user.role
    d_storage = (user.storageUsage / 1024)
    
    #number of content items <=100 is returned by length of items arr
    d_items = len(user.items())
    #print(d_items)
    
    #VT PID is returned by stripping off the _virginiatech
    d_pid = user.username.rsplit("_"+myConfig['agolOrg']['shortName'])[0]
    
    #last access comes from https://developers.arcgis.com/python/guide/accessing-and-managing-users/
    t_last_accessed = time.localtime(user.lastLogin/1000)
    d_lastAccess = "{}/{}/{}".format(t_last_accessed[0], t_last_accessed[1], t_last_accessed[2])
    
    #count of groups this user is a member of
    d_groupCount = len(user.groups)
    
    #Now build a data structure    
    currentUserInfo = {"pid":d_pid,
                        "esriUsername":d_esriUsername,
                        "fullName":d_fullName,
                        "email":d_email,
                        "storage":d_storage,
                        "role":d_role,
                        "lastAccess":d_lastAccess,
                        "groups":d_groupCount,
                        "items":d_items}
    userList.append(currentUserInfo)
    
# iteration done.
# now let's make a dataframe.  We'll use this later.
df = pandas.DataFrame(userList)
df




Now that we have a data structure of user information in memory, we can make administrative decisions based on it.

* Search for a role and grant its members ArcGIS Pro licenses
* Delete "drive by" users
* Identify users not affiliated with the institution using an out-of-band lookup service

In [ ]:
# What are these weird custom roles?   Let's find out
roles = gis.users.roles.all()
for role in roles:
    print(role)

In [ ]:
gis.users.roles.get_role('pH1lPndPVtVbrE6l')

In [ ]:
# Let's get all the users whose role is "New User".
# We're using the pandas query syntax here
df.loc[(df['role'] == 'pH1lPndPVtVbrE6l')]

In [ ]:
# Let's give our new users an ArcGIS Pro license
# For now, we'll use the licenses and entitlements we expect
entitlements = {'ArcGIS Pro': ['geostatAnalystN', 'spatialAnalystN', 'networkAnalystN', 'dataReviewerN',
                               'dataInteropN', 'workflowMgrN', '3DAnalystN', 'desktopAdvN']}
licenses = {lic: gis.admin.license.get(lic) for lic in entitlements}
new_users = gis.users.search("pH1lPndPVtVbrE6l")            
for u in new_users:
    for license_type in entitlements:
        lic = licenses[license_type]
        # THIS IS A DEMO SO WE DON'T ACTUALLY PULL THE TRIGGER
        #lic.assign(username=u.username, entitlements=entitlements[license_type])
        print('{0} entitlements granted to user {1.username}'.format(license_type, u))

## Example 2: Drive by users

In [ ]:
#Let's just look for the users who have no content items or storage, nor are they in any groups.
df.loc[(df['storage'] == 0) & (df['items'] == 0) &(df['groups'] ==0)]

In [ ]:
# Sort by last accessed time.
df.sort_values('lastAccess')

####  I feel reasonably confident we can get rid of users who 
* own no content items
* are members of no groups
* use no storage
* have not logged in for a year

... since if that user logs back in, it will be like they never left.

In [ ]:
# SO we nuke them from orbit
deleteList = df.loc[(df['storage'] == 0) & (df['items'] == 0) &(df['groups'] ==0) &(df['lastAccess'].str[:4] != '2018')]
deleteList

In [ ]:
for index, row in deleteList.iterrows():
    sUserToDelete = df['esriUsername'].values[index]
    print ("Deleting " + sUserToDelete +"...")
    #Here we would simply call
    #userToDelete = gis.users.search(sUserToDelete, max_users=1)
    #userToDelete.delete()

## Out of band affiliation check
Since ArcGIS Online cannot pull extended attributes from a SAML identity provider beyond username and email, we developed a web service to perform a check for "Active" status for any username we provide it.  

NOTE that this does not translate outside of the Virginia Tech context; we developed a custom solution for this.  
Your custom web services can be integrated... in orgConfig.json, provide values for the "authService" group:
* URL
* authKey or password to access the service
* value to be returned if the user is a valid member

In [ ]:
userList = []
users = gis.users.search(max_users=20)

# Wrap the call to the REST service in a function
def isActiveVT(pid):
    result = False
    url = myConfig['authService']['url']
    params = {'authkey':myConfig['authService']['password'],'pid':pid}
    r = requests.post(url,data=params)
    if(r.text==myConfig['authService']['valueIfTrue']):
        result=True
    return result

for user in users:
    d_esriUsername = user.username
    d_fullName = user.fullName
    d_email = user.email
    d_pid = user.username.rsplit("_"+myConfig['agolOrg']['shortName'])[0]

    # Call the custom REST service 
    d_active = "false"
    if(isActiveVT(d_pid)):
        d_active = myConfig['authService']['valueIfTrue']
    
    currentUserInfo = {"pid":d_pid,
                        "esriUsername":d_esriUsername,
                        "fullName":d_fullName,
                        "email":d_email,
                        "active":d_active}
    userList.append(currentUserInfo)
    
# iteration done.
# now let's make a dataframe.  We'll use this later.
df = pandas.DataFrame(userList)
df

In [ ]:
# So in this situation we would want to generate a list of users who need to be deprovisioned.  
# We could send them an automatic e-mail...


# Options for running Python API code in production
* Interactively 
 * Jupyter noteboook
 * Your preferred Python IDE
 * Command line
* Event driven
 * AWS Lambda
* Recurring
 * AWS Lambda
 * cron job/ scheduled task

![lambda](lambda.png)

# Link to this Presentation and Code
![qr](qr.png)
https://github.com/sspeery/esri-seuc-2018 



# Presenter Contact Information
>*Seth Peery*  
>    Senior GIS Architect  
>    Enterprise GIS (0214)  
>        1700 Pratt Drive  
>        Blacksburg, VA 24061  
>    (540) 231-2178  
>    sspeery@vt.edu   
>    http://gis.vt.edu